In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
%matplotlib inline

In [9]:
df = pd.read_excel("../data/adminsrvad.sa.gov.audfsrootusersssaservdeliveryemployeeservmoulet01my-documentsopen-data-2016pr.xlsx", engine="openpyxl", skiprows=4)

df = df.dropna(axis=1, how="all")
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns = ['Unnamed: 0'])

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.head()

,Agency Name,Jun - 15,Jul - 15,Aug - 15,Sep - 15,Oct - 15,Nov - 15,Dec - 15,Jan - 16,Feb - 16,Mar - 16,Apr - 16,May - 16
0,Attorney-General's Department,2317,2419,2029,2418,2122,1939,2242,1551,1753,2076,2226,2129
1,Defence SA,227,203,156,197,184,164,214,139,141,201,195,165
2,Department for Communities and Social Inclusion,22375,30473,17933,160025,32771,23067,22891,28252,47224,21214,31145,19598
3,Department for Correctional Services,3545,4162,3255,4058,4323,3716,3823,3386,3566,4065,4432,4341
4,Department for Education and Child Development,21929,21294,19113,19484,22789,20315,19418,16623,18526,22453,18421,23140


In [12]:
df.describe()

,Jun - 15,Jul - 15,Aug - 15,Sep - 15,Oct - 15,Nov - 15,Dec - 15,Jan - 16,Feb - 16,Mar - 16,Apr - 16,May - 16
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,7228.666667,7259.629630,6164.074074,12135.629630,7447.888889,6862.740741,6862.259259,5503.925926,7168.703704,6966.000000,7002.814815,7237.703704
std,7990.204094,8383.442964,7058.970223,30454.206706,8842.267401,7739.442489,7872.997467,7027.598791,10342.653791,7777.697247,8182.767963,8149.050458
min,81.000000,99.000000,40.000000,44.000000,33.000000,45.000000,42.000000,28.000000,38.000000,35.000000,41.000000,50.000000
25%,2047.000000,2023.000000,1442.500000,1709.500000,1822.000000,1658.500000,1709.000000,1327.500000,1428.500000,1699.500000,1828.500000,1872.500000
50%,3545.000000,3446.000000,2753.000000,3400.000000,3299.000000,3377.000000,3389.000000,2625.000000,2862.000000,3556.000000,3214.000000,3525.000000
75%,8673.500000,8951.500000,7531.000000,9071.500000,9693.500000,8781.000000,8304.500000,6570.500000,7968.500000,9082.000000,9137.000000,9372.500000
max,25047.000000,30473.000000,23184.000000,160025.000000,32771.000000,23425.000000,25975.000000,28252.000000,47224.000000,24930.000000,31145.000000,24827.000000


In [14]:
df_long = pd.melt(
    df,
    id_vars=['Agency Name'],
    var_name='Month',
    value_name='Invoices Paid'
)

In [16]:
# Remove commas and convert into numeric
df_long['Invoices Paid'] = (
    df_long['Invoices Paid']
    .astype(str)
    .str.replace(",", "", regex = 'false')
)

df_long['Invoices Paid'] = pd.to_numeric(df_long['Invoices Paid'], errors = 'coerce')

# Convert Month to dateTime
df_long['Month'] = pd.to_datetime(df_long['Month'], format = "%b - %y", errors = 'coerce')

# Drop rows with invalid months and NaNs
df_long = df_long.dropna(subset=['Month', 'Invoices Paid'])
df_long = df_long.sort_values(by="Month")
df_long.head()

,Agency Name,Month,Invoices Paid
0,Attorney-General's Department,2015-06-01,2317
26,TAFE SA,2015-06-01,3380
25,South Australian Tourism Commission,2015-06-01,592
24,South Australian Housing Trust,2015-06-01,25047
23,South Australian Fire and Emergency Services C...,2015-06-01,3466
